# Chapter 8 Applying Machine Learning to Sentiment Analysis

## 1 IMDB 리뷰 데이터 호출하기
50,000개의 데이터 호출하기 

http://ai.stanford.edu/~amaas/data/sentiment/

In [1]:
import pyprind, os
import pandas as pd

# 파일 불러는 폴더명 지정
basepath = './data/'

# 막대 상태바 (50,000번의 반복으로 초기화 된다)
pbar = pyprind.ProgBar(50000)
# labels = {'pos': 1, 'neg': 0} 로 폴더명에 따라 0과 1로 labeling 한다
labels = {'pos': 1, 'neg': 0}


In [2]:
# 위 설정에 따라 파일을 불러오기
df = pd.DataFrame()

# for s in ('test', 'train'):
#     for l in ('pos', 'neg'):
#         path = os.path.join(basepath, s, l)
#         for file in os.listdir(path):
#             with open(os.path.join(path, file), 'r', encoding='utf-8') as infile:
#                 txt = infile.read()
#             df = df.append([[txt, labels[l]]], ignore_index=True)
#             pbar.update()
# df.columns = ['review', 'sentiment'] # 불러온 뒤 컬럼명 지정하기

In [3]:
# import numpy as np

# # index 를 무작위로 섞는다.
# np.random.seed(0)
# df = df.reindex(np.random.permutation(df.index))

# # 테이블을 CSV로 저장한다
# df.to_csv('./data/movie_data.csv', index=False)

In [4]:
import pandas as pd

df = pd.read_csv('./data/movie_data.csv')
df.head(3)

,review,sentiment
0,"More entertaining than all the gay orgies in ""...",1
1,This Game is a good looking First Person Shoot...,0
2,"It was a painful experience, the whole story i...",0


## 2 Bag of Word Model 소개
a,b 를 1,2 로 변환하듯, Text 를 '수치형 벡터'로 표현한다

### 01 Word to Feature Vector
단어를 피처백터로 변환

In [5]:
# CountVectorizer() 단어의 갯수를 자동으로 Count 한다 (대소문자 무관)
import numpy as np
docs = np.array(['The Sun is shining', 'The weather is sweet',
        'The sun is shining, the weather is sweet, and one and one is two'])

In [6]:
# 수집한 단어들을 자동으로 Count 한다
from sklearn.feature_extraction.text import CountVectorizer
# Uni-Gram 모델을 생성 : ngram_range=(1, 1)  
# cf) ngram_range=(2, 2) 로 바꾸면 2 Gram 을 설정가능
count = CountVectorizer(ngram_range=(1,1))
bag = count.fit_transform(docs)

In [7]:
print(count.vocabulary_)

{'the': 6, 'sun': 4, 'is': 1, 'shining': 3, 'weather': 8, 'sweet': 5, 'and': 0, 'one': 2, 'two': 7}


In [8]:
# 정수 인덱스에 매핑된 Python 사전에 저장되어 있다
# CountVectorizer() 단어집의 사전 item에 저장된 정수에 대응하는 Matrix를 출력
print(bag.toarray())

[[0 1 0 1 1 0 1 0 0]
 [0 1 0 0 0 1 1 0 1]
 [2 3 2 1 1 1 2 1 1]]


In [9]:
np.set_printoptions(precision=2)

### 02 용어 빈도수의 역수를 활용한, 단어의 관련성 평가
단어의 빈도수 : Term Frequency-inversed Document

In [ ]:
# 'CountVectorizer'의 'Raw'항의 빈도수를 입력값으로, 'T-fid-f-Transformer'를 생성
# Feature Vector 에서 '빈번하게 등장하는 단어'들의 '가중치를 낮추는데 사용 (불용어 사전 해당용어들)
# 문서내 빈도수의 역수의 곱

In [10]:
# TfidfTransformer 에서는 자동으로 '정규화'를 거친다
from sklearn.feature_extraction.text import TfidfTransformer

# norm='l2' : 벡터 l2표준으로 나눠서, 길이가 1인 벡터를 반환한다
tfidf = TfidfTransformer(use_idf=True, norm='l2', smooth_idf=True)
print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

[[ 0.    0.43  0.    0.56  0.56  0.    0.43  0.    0.  ]
 [ 0.    0.43  0.    0.    0.    0.56  0.43  0.    0.56]
 [ 0.5   0.45  0.5   0.19  0.19  0.19  0.3   0.25  0.19]]


In [ ]:
# 0.56 빈도의 단어는 'is'로써 가장 큰 빈도를 갖는다

In [11]:
tf_is = 3
n_docs = 3
idf_is = np.log((n_docs+1) / (3+1))
tfidf_is = tf_is * (idf_is + 1)
print('tf-idf of term "is" = %.2f' % tfidf_is)

tf-idf of term "is" = 3.00


In [16]:
print('tf-idf of term "is" = %.9f' % idf_is)

tf-idf of term "is" = 0.000000000


In [12]:
tfidf = TfidfTransformer(use_idf=True, norm=None, smooth_idf=True)
raw_tfidf = tfidf.fit_transform(count.fit_transform(docs)).toarray()[-1]
raw_tfidf 

array([ 3.39,  3.  ,  3.39,  1.29,  1.29,  1.29,  2.  ,  1.69,  1.29])

In [13]:
l2_tfidf = raw_tfidf / np.sqrt(np.sum(raw_tfidf**2))
l2_tfidf

array([ 0.5 ,  0.45,  0.5 ,  0.19,  0.19,  0.19,  0.3 ,  0.25,  0.19])

### 03 텍스트 데이터의 정제
<\html> tag 들을 제거하기

In [17]:
df.loc[0, 'review'][-50:]  # index 50번의 review 데이터를 수집한다

' and poetic. It truly is an exercise in poor taste'

In [20]:
# 불필요한 기호들을 제거하는 filter 함수
# HTML tag들을 제거한다
import re
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    return text

preprocessor(df.loc[0, 'review'][-50:])

' and poetic it truly is an exercise in poor taste'

In [21]:
preprocessor("</a>This :) is :( a test :-)!")

'this is a test :) :( :)'

In [22]:
# 리뷰 데이터중 불필요한 기호들을 삭제하는 전처리 작업을 진행한다
df['review'] = df['review'].apply(preprocessor)

### 04 문서를 Tokeize로 처리하기
Sentence 내부의 text를 개별 요소로 분리한다

In [ ]:
# Line no 21
# NLTK를 설치후 진행을 해야 한다
# (이부분은 설치에 시간이 오래 걸리므로 유의 할 것!!)

In [23]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [27]:
# https://packages.debian.org/experimental/amd64/openjdk-7-jdk/download
# Java 7 을 다운 받아야 Kkma등이 실행가능하다
# sudo apt-get install g++ openjdk-7-jdk python-dev python3-dev
from konlpy.tag import Kkma

ModuleNotFoundError: No module named 'jpype'